In [3]:
# from keras.api.datasets import mnist
# import sklearn.preprocessing
import numpy as np
from nn.utils import get_indices, col2im, im2col, add_padding

In [4]:
# (train_x, train_y), (test_x, test_y) = mnist.load_data()

# train_x = train_x / 255
# test_x = test_x / 255

# train_x.resize(60000, 784)
# test_x.resize(10000, 784)

# encoder = sklearn.preprocessing.OneHotEncoder()
# train_y = encoder.fit_transform(train_y[..., None]).toarray()
# test_y = encoder.fit_transform(test_y[..., None]).toarray()

# train_x.shape, train_y.shape

In [5]:
# sample = train_x[0].reshape((1, 28, 28))

# sample = np.random.randint(1, 25, size=(3, 4, 4))
sample = np.arange(1, 49).reshape((3, 4, 4))

display(sample)
weights = np.random.randint(1, 10, size=(3, 3, 2, 2))
# display(weights)

x_col = im2col(sample, (2, 2), (1, 1), (0, 0))

x_col

array([[[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12],
        [13, 14, 15, 16]],

       [[17, 18, 19, 20],
        [21, 22, 23, 24],
        [25, 26, 27, 28],
        [29, 30, 31, 32]],

       [[33, 34, 35, 36],
        [37, 38, 39, 40],
        [41, 42, 43, 44],
        [45, 46, 47, 48]]])

array([[ 1,  2,  3,  5,  6,  7,  9, 10, 11],
       [ 2,  3,  4,  6,  7,  8, 10, 11, 12],
       [ 5,  6,  7,  9, 10, 11, 13, 14, 15],
       [ 6,  7,  8, 10, 11, 12, 14, 15, 16],
       [17, 18, 19, 21, 22, 23, 25, 26, 27],
       [18, 19, 20, 22, 23, 24, 26, 27, 28],
       [21, 22, 23, 25, 26, 27, 29, 30, 31],
       [22, 23, 24, 26, 27, 28, 30, 31, 32],
       [33, 34, 35, 37, 38, 39, 41, 42, 43],
       [34, 35, 36, 38, 39, 40, 42, 43, 44],
       [37, 38, 39, 41, 42, 43, 45, 46, 47],
       [38, 39, 40, 42, 43, 44, 46, 47, 48]])

In [6]:
weights.reshape((3, 12))

array([[3, 7, 2, 4, 6, 3, 5, 5, 1, 7, 5, 7],
       [5, 8, 7, 3, 9, 4, 5, 9, 9, 1, 3, 4],
       [9, 2, 6, 9, 5, 4, 6, 3, 2, 9, 3, 9]])

In [7]:
bias = np.random.randint(1, 10, size=(3, 1))
bias

array([[1],
       [8],
       [8]])

In [8]:
display(weights.reshape((3, -1)) @ x_col + bias)
# (weights.reshape((-1, 12)) @ x_col).reshape((-1, 3, 3))

array([[1145, 1200, 1255, 1365, 1420, 1475, 1585, 1640, 1695],
       [1204, 1271, 1338, 1472, 1539, 1606, 1740, 1807, 1874],
       [1279, 1346, 1413, 1547, 1614, 1681, 1815, 1882, 1949]])

In [23]:
def conv2d(x, weights, bias, strides, padding):
    window = weights.shape[-2:]
    
    num_filters = weights.shape[0]
    n_H = int((x.shape[1] + 2 * padding[0] - window[0]) / strides[0]) + 1
    n_W = int((x.shape[2] + 2 * padding[1] - window[1]) / strides[1]) + 1
    
    x_col = im2col(x, window, strides, padding)
    
    weights_flatted = weights.reshape((num_filters, -1))
    bias_flatted = bias.reshape((-1, 1))
    
    convolved = (weights_flatted @ x_col) + bias_flatted
    convolved = convolved.reshape((num_filters, n_H, n_W))
    return convolved, x_col

In [24]:
weights = np.random.randint(1, 10, size=(3, 3, 2, 2))

In [30]:
output, x_col = conv2d(sample, weights, bias, (1, 1), (0, 0))

In [62]:
# error = np.ones_like(output)
error = np.random.randint(1, 40, size=output.shape)

In [61]:
def backward(error, x_col, weights, bias, strides, padding, prev_shape):
    window = weights.shape[-2:]
    
    num_filters = weights.shape[0]
    
    error_reshaped = error.reshape((num_filters, -1))
    
    de_dW = (error_reshaped @ x_col.T).reshape(weights.shape)
    de_dB = np.sum(error, axis=(1, 2))
    
    weights_flatted = weights.reshape((num_filters, -1))
    error_col = weights_flatted.T @ error_reshaped
    de_dX = col2im(error_col, prev_shape, window, strides, padding)
    return de_dX, de_dW, de_dB

In [63]:
backward(error, x_col, weights, bias, (1, 1), (0, 0), sample.shape)

(array([[[  51.,  449.,  517.,  144.],
         [ 287., 1002., 1216.,  464.],
         [ 665., 1258.,  987.,  344.],
         [ 393.,  697.,  462.,  214.]],
 
        [[ 296.,  953.,  923.,  225.],
         [ 629., 1573., 1727.,  736.],
         [ 946., 1776., 1322.,  461.],
         [ 525.,  851.,  454.,  250.]],
 
        [[ 213.,  683.,  856.,  387.],
         [ 617., 1768., 1694.,  603.],
         [ 814., 1646., 1373.,  561.],
         [ 560.,  917.,  488.,  203.]]]),
 array([[[[1274, 1488],
          [2130, 2344]],
 
         [[4698, 4912],
          [5554, 5768]],
 
         [[8122, 8336],
          [8978, 9192]]],
 
 
        [[[1038, 1211],
          [1730, 1903]],
 
         [[3806, 3979],
          [4498, 4671]],
 
         [[6574, 6747],
          [7266, 7439]]],
 
 
        [[[1013, 1195],
          [1741, 1923]],
 
         [[3925, 4107],
          [4653, 4835]],
 
         [[6837, 7019],
          [7565, 7747]]]]),
 array([214, 173, 182]))